# 결정트리 튜닝을 위한 교차 검증과 그리드 서치

In [ ]:
import pandas as pd
wine = pd.read_csv('./wine.csv')

# 클래스 열을 타겟으로 사용하고 나머지 열은 특성 배열에 저장

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)

X_sub_train, X_val, y_sub_train, y_val = train_test_split(X_train, y_train, test_size=.2, random_state=42)

print(X_sub_train.shape, X_val.shape)

In [ ]:
# 결정트리 모델 


In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt =DecisionTreeClassifier(random_state=42)
dt.fit(X_sub_train, y_sub_train)
print(dt.score(X_sub_train, y_sub_train))
print(dt.score(X_val, y_val))

## 교차 검증

In [ ]:
# 교차 검증

In [ ]:
# 분류기를 지정한 교차 검증

In [ ]:
# 훈련 세트를 섞은 후 10폴드 교차 검증을 수행하려면


##  결정 트리 하이퍼 파라미터 튜닝을 위한 교차검증과 그리드 서치 하이퍼 파라미터 튜닝

#### 기본 매개 변수를 사용한 결정 트리 모델에서 min_impurity_decrease매개변수의 최적값 찾기

In [ ]:
from sklearn.model_selection import GridSearchCV
params={'min_impurity_decrease':[0.0001, 0.0002,0.0003,0.0004,0.0005]}

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
# 결정트리 클래스의 객체를 생성하자마자 바로 전달
# cv 기본값 5
# min_impurity_decrese값마다 5번의 교차검증 25개의 모델을 훈련
# n_jobs= 병렬 실행에 사용할 CPU 코어 수(기본값 1, 모든 코어 사용: -1)

gs.fit(X_train, y_train)

In [ ]:
# best_params_

In [ ]:
#각 매개변수에서 사용한 교차 검증의 평균 점수

In [ ]:
#np.argmax() - 가장 큰 값의 인덱스 추출
# 위에서 구한 인덱스를 사용해 params 키에 저장된 매개변수 출력

* 복잡한 매개변수 조합의 GridSearchCV

#### 결정 트리: 불순도 감소 최소량(min_impurity_decrease), 트리깊이(max_depth), 노드를 나누기 위한최소샘플(min_samples_split) 찾기

In [ ]:
params = {"min_impurity_decrease": np.arange(0.0001, 0.001, 0.0001),
         'max_depth': range(5, 20, 1), 
         'min_samples_split': range(2, 100, 10)}
# 9 * 15 * 10 = 1350개의 경우의 수 * 5폴드 => 6750개의 모델 수


##  랜덤 서치

# [실습]  GridSearchCV 실습 - 사용자 행동 인식 데이터세트 결정트리

* 사용자 행동 인식(Human Activity Recognition) 데이터로 동작 예측

https://archive.ics.uci.edu/ml/datasets/Human+Activity+Recognition+Using+Smartphones

In [ ]:
# 출처: UCI Machine Learning Repository
# 30명의 사람에게 스마트폰 센서를 장착한 뒤 사람의 동작과 관련된 여러가지 피처를 수집
# 수집된 피처세트를 기반으로 결정트리를 이용해 어떠한 동작인지 예측
# walking, walking_upstairs, walking_downstairs, stting, standing, laying

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# features.txt 파일에는 피처 이름 index와 피처명이 공백으로 분리되어 있음. 이를 DataFrame으로 로드.
feature_name_df = pd.read_csv('./human_activity/features.txt',sep='\s+',
                        header=None,names=['column_index','column_name'])

# 피처명 index를 제거하고, 피처명만 리스트 객체로 생성한 뒤 샘플로 10개만 추출
feature_name = feature_name_df.iloc[:, 1].values.tolist()
print('전체 피처명에서 10개만 추출:', feature_name[:10])


In [ ]:
#중복된 피처명 확인

In [ ]:
def get_new_feature_name_df(old_feature_name_df):
    feature_dup_df = pd.DataFrame(data=old_feature_name_df.groupby('column_name').cumcount(),
                                  columns=['dup_cnt'])
    feature_dup_df = feature_dup_df.reset_index()
    new_feature_name_df = pd.merge(old_feature_name_df.reset_index(), feature_dup_df, how='outer')
    new_feature_name_df['column_name'] = new_feature_name_df[['column_name', 'dup_cnt']].apply(lambda x : x[0]+'_'+str(x[1]) 
                                                                                         if x[1] >0 else x[0] ,  axis=1)
    new_feature_name_df = new_feature_name_df.drop(['index'], axis=1)
    return new_feature_name_df

In [ ]:
# train feature dataset, label data set, test용 feature dataset , label data set

* 결정트리를 사용해 동작 예측 분류 수행

* 모든 매개 변수를 기본(디폴트) 값으로 설정한 후 결정트리 수행

* GridSearchCV를 이용해 max_depth값을 변화시키면서 예측 성능 확인 (2분 ~)

In [ ]:
# 깊어진 트리는 검증 데이터 세트에서는 과적합, 성능 저하

In [ ]:
# 테스트 데이터세트에서 max_depth의 변화에 따른 값 확인

In [ ]:
# max_depth, min_samples_split 매개변수를 변경하면서 GridSearchCV수행

In [ ]:
# 테스트 데이터세트에 최적 하이퍼파라미터 작용 

In [ ]:
# 각 피처별 중요도 표현